### 3D City Models from Volunteered Public Data

**This notebook will produce an interactive 3D Building Model visualization** *- via [pydeck](https://deckgl.readthedocs.io/en/latest/) -* **of multiple campuses/sites (via an osm [relation=site](https://wiki.openstreetmap.org/wiki/Relation:site)) which you can navigate and query.**

In [ ]:
#load the magic

%matplotlib inline
import os
from pathlib import Path
import requests

import overpass
import osm2geojson

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
shapely.speedups.disable()
from shapely.geometry import Polygon, shape, mapping
import json
import geojson

from osgeo import ogr

from openlocationcode import openlocationcode as olc

import pydeck as pdk

**Harvest [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap)** - Query the [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) from within Jupyter and convert to .geojson.

<div class="alert alert-block alert-info"><b>Set an area-of-interest:</b> 
    
This is done: larger area -> focus area or State (Province) -> Village (campus)
</div>

In [2]:
large = 'Western Cape'
focus = 'Cape Peninsula University of Technology'
osm_type = 'way'

In [3]:
query = """
[out:json][timeout:30];
// --when areas have duplicate names given the world has a limited amount of uniquely named places
(area[name='{0}'] ->.b;
    // -- target area ~ can be way or relation
    {1}(area.b)[operator='{2}'];
    map_to_area -> .a;
        // I want all buildings
        way['building'](area.a);
        // and relation type=multipolygon ~ to removed courtyards from buildings
        relation["building"]["type"="multipolygon"](area.a);
    );
out body;
>;
out skel qt;
""".format(large, osm_type, focus)

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
#rr = r.read()
gj = osm2geojson.json2geojson(r.json())

In [4]:
# have a look at a random feature
gj['features'][13]

{'type': 'Feature',
 'properties': {'type': 'way',
  'id': 517956197,
  'tags': {'addr:housename': 'Suddenburg Residence',
   'building': 'dormitory',
   'building:levels': '4',
   'residential': 'university'}},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[18.4329649, -33.9312172],
    [18.4328456, -33.9314264],
    [18.4324701, -33.931279],
    [18.4325768, -33.9310918],
    [18.4323137, -33.9309886],
    [18.432499, -33.9306635],
    [18.4331286, -33.9309105],
    [18.4330091, -33.9311203],
    [18.4329001, -33.9310775],
    [18.4329594, -33.9309735],
    [18.4328219, -33.9309195],
    [18.4327592, -33.9310295],
    [18.4326458, -33.930985],
    [18.4327219, -33.9308512],
    [18.4325498, -33.9307838],
    [18.432428, -33.9309979],
    [18.4326881, -33.9310998],
    [18.4326747, -33.9311234],
    [18.4327069, -33.931136],
    [18.4326593, -33.9312197],
    [18.4327889, -33.9312705],
    [18.4328331, -33.9311928],
    [18.4328573, -33.9312023],
    [18.4328701, -33.93118],
   

<div class="alert alert-block alert-warning"><b>Calculate building height:</b> 

We assume a building level is 2.8 meters high and add another 1.3 meters (to account for the roof) and create a new attribute height.</div>

In [5]:
storeyheight = 2.8

#-- iterate through the list of buildings and create GeoJSON features rich in attributes
footprints = {
    "type": "FeatureCollection",
    "features": []
    }

for i in gj['features']:
    f = {
    "type" : "Feature"
    }
    # at a minimum we only want building:levels tagged, exclude building nodes and no tags
    if i['properties']['type'] != 'node' and 'tags' in i['properties'] \
    and 'building:levels' in i['properties']['tags'] is not None:

        f["properties"] = {}
        for p in i["properties"]:             
        #-- store OSM attributes and prefix them with osm_
            f['properties']['osm_id'] = i['properties']['id']
        
            f["properties"]["osm_%s" % p] = i["properties"][p]
            if 'amenities' in i['properties']:
                f['properties']['osm_tags']['amenity'] = i['properties']['amenities']
        osm_shape = shape(i["geometry"])
        #-- a few buildings are not polygons, rather linestrings. This converts them to polygons
        #-- rare, but if not done it breaks the code later
        if osm_shape.type == 'LineString':
            osm_shape = Polygon(osm_shape)
        #-- and multipolygons must be accounted for
        elif osm_shape.type == 'MultiPolygon':
            osm_shape = Polygon(osm_shape[0])
        #-- convert the shapely object to geojson
        f["geometry"] = mapping(osm_shape)

        #-- calculate the height and store it as an attribute
        f["properties"]['height'] = float(i["properties"]['tags']['building:levels']) * storeyheight + 1.3  
        #plus code
        p = osm_shape.representative_point()
        f["properties"]["plus_code"] = olc.encode(p.y, p.x, 11)
            
        footprints['features'].append(f)


#-- store the data as GeoJSON
with open('./data/fp_cput_j.geojson', 'w') as outfile:
    json.dump(footprints, outfile)

**[pydeck](https://deckgl.readthedocs.io/en/latest/) needs an area and a center**, harvest from osm.

In [96]:
query = """
[out:json][timeout:30];
area[name='{0}']->.a;
//gather results
(
  // query part for: “university”
  {1}['operator'='{2}'](area.a);
);
//print results
out body;
>;
out skel qt;
""".format(large, osm_type, focus)

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
#rr = r.read()
area = osm2geojson.json2geojson(r.json())

In [105]:
#read into .gpd
gdf = gpd.GeoDataFrame.from_features(area['features'])

#-- some relation aoi's are many relations ~ extract the 'place' 
if osm_type == 'relation' and len(gdf) > 1:
    gdf.dropna(subset = ["tags"], inplace=True)
    for i, row in gdf.iterrows():
        if row.tags != None and row.tags != np.nan and 'place' in row.tags:
            focus = row
            
    trim = pd.DataFrame(focus)
    trim = trim.T
    gdf = gpd.GeoDataFrame(trim, geometry = trim['geometry'])
        
#gdf = gdf.set_crs(4326)

# -- get the location for pydeck
#[xy] = gdf.centroid
xy = gdf.geometry[0].centroid.xy    #--- I'm choosing this because the 'Main Campus' is here
#bbox = [gdf.total_bounds[0], gdf.total_bounds[1], 
        #gdf.total_bounds[2], gdf.total_bounds[3]]
bbox = [gdf.geometry[0].bounds[0], gdf.geometry[0].bounds[1], 
        gdf.geometry[0].bounds[2], gdf.geometry[0].bounds[3]]

***~ In order to make the most of the semantic data we need to extract the `osm_tags` from the dictionary: and add it as `tooltips` to the visualization.***

<div class="alert alert-block alert-success"><b>Building Stock:</b> To differentiate a school, housing, retail, healthcare and community focused facilities (library, municipal office, community centre) we color the buildings - we harvest the osm tags [amenity and building type] directly.</div>

In [112]:
data = './data/fp_cput_j.geojson'
json = pd.read_json(data)
build_df = pd.DataFrame()

# Parse the geometry out to Pandas
build_df["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])
build_df["height"] = round(json["features"].apply(lambda row: row["properties"]["height"]), 1)
build_df["plus_code"] = json["features"].apply(lambda row: row["properties"]["plus_code"])


#we want to display data so extract values from the dictionary 
build_df["id"] = json["features"].apply(lambda row: row["properties"]["osm_id"])
#build_df = build_df.loc[(build_df.id != 13076003) & (build_df.id != 12405081)]

build_df["tags"] = json["features"].apply(lambda row: row["properties"]["osm_tags"])
build_df['level'] = build_df['tags'].apply(lambda x: x.get('building:levels'))
build_df['name'] = build_df['tags'].apply(lambda x: x.get('addr:housename'))

In [113]:
# have a look at the building type and amenities available
#build_df['combine'].unique()
build_df.head(2)

,coordinates,height,plus_code,id,tags,level,name
0,"[[[18.424828, -33.9324079], [18.4244374, -33.9...",9.7,4FRW3C8F+XW6,44559793,"{'addr:city': 'Cape Town', 'addr:postcode': '8...",3,None
1,"[[[18.4282547, -33.9289116], [18.4283323, -33....",9.7,4FRW3CCH+F9H,141373715,"{'addr:city': 'Cape Town', 'addr:housename': '...",3,Cape Peninsula University of Technology (E-Lea...


*I want additional visual effects to show the potential and power of **3D City Models**; namely: greenspaces, parking and waterways (streams). We get this from OpenStreetMap as well.*

In [38]:
query = """
[out:json][timeout:30];
// --when areas have duplicate names given the world has a limited amount of uniquely named places
// --main area
area[name='{0}']->.b;
// -- target area ~ can be way or relation
{1}(area.b)[operator='{2}'];
map_to_area -> .a;
(
  // query
  way["leisure"~'pitch'](area.a);
);
// print results
out body;
>;
out skel qt;
""".format(large, osm_type, focus)

url = "http://overpass-api.de/api/interpreter"
p = requests.get(url, params={'data': query})
#rr = r.read()
green_spaces = osm2geojson.json2geojson(p.json())

query = """
[out:json][timeout:30];
// --when areas have duplicate names given the world has a limited amount of uniquely named places
// --main area
area[name='{0}']->.b;
// -- target area ~ can be way or relation
{1}(area.b)[operator='{2}'];
map_to_area -> .a;
(
// query
  way["natural"~'water'](area.a);
);
// print results
out body;
>;
out skel qt;
""".format(large, osm_type, focus)

url = "http://overpass-api.de/api/interpreter"
w = requests.get(url, params={'data': query})
#rr = r.read()
water_spaces = osm2geojson.json2geojson(w.json())

query = """
[out:json][timeout:30];
// --when areas have duplicate names given the world has a limited amount of uniquely named places
// --main area
area[name='{0}']->.b;
// -- target area ~ can be way or relation
{1}(area.b)[operator='{2}'];
map_to_area -> .a;
(
  // query
  way['parking'](area.a);
);
// print results
out body;
>;
out skel qt;
""".format(large, osm_type, focus)

url = "http://overpass-api.de/api/interpreter"
f = requests.get(url, params={'data': query})
#rr = r.read()
p_spaces = osm2geojson.json2geojson(f.json())

**I also want to show the extensive campus bus service** *---of which only a tiny portion has been captured. If you come across this and want to contribute; please feel free. [CPUT](https://www.cput.ac.za/) might even fund your research project to investigate the effectiveness of its transport network and perhaps refine and improve its routes.*

In [42]:
# the bus route ~~ note we only choose routes with a 'colour' tag
query = """
[out:json][timeout:30];
area[name='{0}'];
// -- target area ~ can be way or relation
// gather results
(
  // query part for: “"bus route"”
  relation["type"="route"]["route"="bus"]['operator'="CPUT Shuttle (HG Travelling Services)"]['colour'](area);
);
// print results
out body;
>;
out skel qt;
""".format(large)

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
#rr = r.read()
r_lines = osm2geojson.json2geojson(r.json())

In [43]:
# have a look at a random bus route
r_lines['features'][0]['properties']

{'type': 'relation',
 'id': 13092009,
 'tags': {'access': 'private',
  'colour': '#34eb4c',
  'duration': '00:20:00',
  'from': 'CPUT Cape Town Campus',
  'name': 'Bus BC1: CPUT Cape Town => CPUT Bellville',
  'network': 'Cape Peninsula University of Technology (CPUT)',
  'operator': 'CPUT Shuttle (HG Travelling Services)',
  'public_transport:version': '2',
  'ref': 'BC1',
  'route': 'bus',
  'to': 'CPUT Belliville Campus',
  'type': 'route',
  'university': 'yes'}}

In [44]:
# extract path and assign colour ~~ so the visualization matches the official documentation

Rgdf = gpd.GeoDataFrame.from_features(r_lines['features'])
Rgdf = Rgdf.explode()
def coords(geom):
    return list(geom.coords)
Rgdf['path'] = Rgdf.apply(lambda row: coords(row.geometry), axis=1)

Rgdf['name'] = Rgdf['tags'].apply(lambda x: x.get('name')\
                                    if isinstance(x, dict) else np.nan)
Rgdf['colour'] = Rgdf['tags'].apply(lambda x: x.get('colour', np.nan)\
                                    if isinstance(x, dict) else np.nan)
Rgdf = Rgdf[Rgdf['colour'].notna()]

def hex_to_rgb(h):
    h = h.lstrip("#")
    #h = h.replace('#', '')
    return tuple(int(h[i : i + 2], 16) for i in (0, 2, 4))

Rgdf["colour"] = Rgdf["colour"].apply(hex_to_rgb)

Rgdf.head(3)

type  ...         colour
0 0  relation  ...  (52, 235, 76)
1 0  relation  ...  (52, 235, 76)
  1  relation  ...  (52, 235, 76)

[3 rows x 7 columns]

In [115]:
## ~ (x, y) - bl, tl, tr, br  ~~ or ~~ sw, nw, ne, se
#area = [[[18.4377, -33.9307], [18.4377, -33.9283], [18.4418, -33.9283], [18.4418, -33.9307]]]
area = [[[bbox[0], bbox[1]], [bbox[0], bbox[3]], 
         [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]

## ~ (y, x)
view_state = pdk.ViewState(latitude=xy[1][0], longitude=xy[0][0], zoom=15.5, max_zoom=19, pitch=72, 
                                   bearing=80)

land = pdk.Layer(
    "PolygonLayer",
    area,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 1],
    #material = True,
    #shadowEnabled = True
)
building_layer = pdk.Layer(
    "PolygonLayer",
    build_df,
    #id="geojson",
    opacity=0.3,
    stroked=False,
    get_polygon="coordinates",
    filled=True,
    extruded=True,
    wireframe=False,
    get_elevation="height",
    get_fill_color="[255, 255, 255]", #255, 255, 255
    #get_fill_color="fill_color",
    get_line_color=[255, 255, 255], #"fill_color",#
    #material = True, 
    #shadowEnabled = True, 
    auto_highlight=True,
    pickable=True,
)
greenspaces_layer =  pdk.Layer(
    "GeoJsonLayer",
    green_spaces,
    opacity=0.5,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color="[14, 140, 58]",
    get_line_color='[14, 140, 58]',
)
p_layer =  pdk.Layer(
    "GeoJsonLayer",
    p_spaces,
    opacity=0.2,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color="[50, 168, 139]",
    get_line_color='[50, 168, 139]',
)

water_layer =  pdk.Layer(
    "GeoJsonLayer",
    water_spaces,
    opacity=0.8,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color="[35, 35, 142]",
    get_line_color='[35, 35, 142]',
)
r_layer = pdk.Layer(
    type="PathLayer",
    data=Rgdf,
    get_color='colour', #'[245, 51, 58]',
    #width_scale=20,
    #width_min_pixels=8,
    get_path="path",
    get_width=5,
    auto_highlight=False, # change to True if route query
    pickable=False, # change to True if route query
)

tooltip = {"html": "<b>Levels:</b> {plus_code} <br/> <b>Name:</b> {name}"}# <br/> <b>id:</b> {id}"}

#change the tooltip to show bus routes and comment out the previous
#tooltip = {"html": "<b>Route:</b> {name} <br/>"}

r = pdk.Deck(layers=[land, building_layer, greenspaces_layer, p_layer, water_layer, r_layer], 
             #views=[{"@@type": "MapView", "controller": True}],
             initial_view_state=view_state,
             map_style = 'dark_no_labels', 
             tooltip=tooltip)
#save
r.to_html("./result/interactiveCPUT-many.html")

**on a laptop without a mouse:**

- `trackpad left-click drag-left` and `-right`;
- `Ctrl left-click drag-up`, `-down`, `-left` and `-right` to rotate and so-on and
- `+` next to Backspace zoom-in and `-` next to `+` zoom-out.

**Now you do your community.** ~ If your area lacks OpenStreetMap data and you want to contribute please follow the [Guide](https://wiki.openstreetmap.org/wiki/Beginners%27_guide).